In [1]:
from NextVisionML import MLContext
from NextVisionML import VarianceFilter, DecisionTreeClfr, OneHotClfr

In [ ]:
#Create an context (Loads data)
mlcontext = MLContext()

In [ ]:
#add feature selection methods
mlcontext.hooks.append(VarianceFilter(mlcontext)) 
#Add data prepocessing methods
mlcontext.train_methods.append(DecisionTreeClfr(mlcontext))
mlcontext.train_methods.append(OneHotClfr(mlcontext))
#select a model type

#start the training process
mlcontext.start_train_process()

In [ ]:
import pandas as pd
import dice_ml
from dice_ml.utils import helpers # helper functions
from sklearn.model_selection import train_test_split

In [ ]:
import hyperopt

In [ ]:
from NextVisionML import load_context

sqlContext, train_with_labels, test_with_labels = load_context()

In [ ]:
test_with_labels

In [ ]:
train_dataset = pd.concat([mlcontext.iter_train_X[2], mlcontext.iter_train_y[2]], axis = 0)
cols = []#["Amb_WindSpeed_Std", "Avg_Precipitation", "Grd_Prod_CosPhi_Avg", "Max_Precipitation", "Min_Precipitation"]
train_dataset = train_dataset.drop(columns=cols)
d = dice_ml.Data(
    dataframe=train_dataset,
    continuous_features= list(train_dataset.drop(columns=["Risk Level"]).columns),
    outcome_name="Risk Level")

# Using sklearn backend
m = dice_ml.Model(model=mlcontext.iter_objs[2]["model"], backend="sklearn")
# Using method=random for generating CFs
exp = dice_ml.Dice(d, m, method="random")

In [ ]:
test_data = mlcontext.iter_test_X[2].drop(columns = cols)

In [ ]:
e1 = exp.generate_counterfactuals(test_data.iloc[:5].copy(), total_CFs=2, desired_class=0)

In [ ]:
e1.visualize_as_dataframe(show_only_changes=False)

In [ ]:
import torch
from sklearn.metrics import balanced_accuracy_score

In [ ]:
class SimpleNetSoftmax(torch.nn.Module):
    def __init__(self, input_size, output_size, layer_count):
        super(SimpleNetSoftmax, self).__init__()
        self.call_super_init
        self.fc = torch.nn.ModuleList()
        self.lc = layer_count
        for _ in range(layer_count-1):
            self.fc.append(torch.nn.Linear(input_size, input_size*2))
            input_size*=2
        self.fc.append(torch.nn.Linear(input_size, output_size))
        #self.sigmoid = torch.sigmoid

    def forward(self, X):
        y = X
        for layer in self.fc:
            y = layer(y)
        y = torch.softmax(y, dim=1)
        return y
    
i=0
train_X = torch.tensor(mlcontext.iter_train_X[i].astype('float32').values)
min, _ = train_X.min(dim=0)
max, _ = train_X.max(dim=0)
normalized_X = (train_X - min) / (max-min + 1)

normalized_X = normalized_X

train_y = torch.tensor(mlcontext.iter_objs[i]["integer_encoded_train_y"])
random_seed = 5521
input_size = len(mlcontext.iter_train_X[i].columns)
num_classes = 5
layer_count = 4
num_epochs = 5  
one_hot_encoded = torch.nn.functional.one_hot(train_y.to(torch.int64), num_classes).float()
torch.manual_seed(random_seed)
net = SimpleNetSoftmax(input_size, num_classes, layer_count)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.005)

#Training loop
for epoch in range(num_epochs):
    outputs = net(normalized_X)
    loss = criterion(outputs, one_hot_encoded)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

#Evluation     
test_X = torch.tensor(mlcontext.iter_test_X[i].astype('float32').values)
min, _ = test_X.min(dim=0)
max, _ = test_X.max(dim=0)
test_X_n = (test_X - min) / (max-min + 1)

test_X_n = test_X_n

pred = net(test_X_n)
_ , pred = torch.max(pred, 1) #Reverse One Hot
#pred_mem = pred.tolist()
import pandas as pd
from alibi.confidence import TrustScore
import numpy as np
def get_trust_scores(train_X, train_y, test_X, test_pred):  
    #test_pred = pd.DataFrame(test_pred)  
    #class_mapping = {"low": 0, "low-med": 1, "medium": 2, "med-high": 3, "high": 4}
    #train_y= train_y["Risk Level"].map(class_mapping)
    #test_pred = test_pred[0].map(class_mapping)
    ts = TrustScore()
    ts.fit(train_X, train_y, classes=5)
    #classes = {"low", "low-med", "medium", "med-high", "high"}
    score, closest_class = ts.score(test_X, test_pred, k=2)
    return score


#df_ = pd.DataFrame(train_y.numpy().reshape(-1,1))
#df_ = df_.rename(columns={0: "Risk Level"})
t = get_trust_scores(normalized_X.numpy(), pd.DataFrame(train_y.numpy().reshape(-1,1).astype(dtype = np.int64))[0], test_X_n.numpy(), pred.numpy().reshape(-1,1))
print(t)

In [ ]:
len(t)

In [ ]:
from alibi.confidence import TrustScore
def get_trust_scores(train_X, train_y, test_X, test_pred):    
    ts = TrustScore()
    ts.fit(train_X, train_y, classes=5)
    #classes = {"low", "low-med", "medium", "med-high", "high"}
    score, closest_class = ts.score(test_X, test_pred, k=2)
    return score
ts = get_trust_scores(train_X.numpy(), train_y.numpy(), test_X_n.numpy(), pred)

In [ ]:
i=0
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
integer_encoded_train_y = label_encoder.fit_transform(mlcontext.iter_train_y[i].values.ravel())
integer_encoded_test = label_encoder.transform(mlcontext.iter_test_y[i].values.ravel())

In [ ]:
integer_encoded_test

In [ ]:
mlcontext.iter_train_y[0].columns

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from NextVisionML.util import update_object_attributes
from alibi.confidence import TrustScore
from sklearn.metrics import balanced_accuracy_score
import pandas as pd

def get_trust_scores(train_X, train_y, test_X, test_pred):  
    test_pred = pd.DataFrame(test_pred)  
    class_mapping = {"low": 0, "low-med": 1, "medium": 2, "med-high": 3, "high": 4}
    train_y= train_y["Risk Level"].map(class_mapping)
    test_pred = test_pred[0].map(class_mapping)
    ts = TrustScore()
    ts.fit(train_X, train_y, classes=5)
    #classes = {"low", "low-med", "medium", "med-high", "high"}
    score, closest_class = ts.score(test_X, test_pred, k=2)
    return score
i=0
dtc = DecisionTreeClassifier(
    max_depth = 3,
    min_samples_leaf = 2,
    random_state = 5,
    max_features = 3
)
dtc.fit(mlcontext.iter_train_X[i].values, mlcontext.iter_train_y[i].values)
#self.mlContext.iter_objs[i]["model"]["dtc"] = dtc
eval_predict = dtc.predict(mlcontext.iter_test_X[i].values) 
balanced_accuracy = balanced_accuracy_score(mlcontext.iter_test_y[i], eval_predict) 

mlcontext.iter_objs[i]["dtc_pred"] = eval_predict
mlcontext.iter_objs[i]["dtc_balanced_accuracy"] = balanced_accuracy
mlcontext.iter_objs[i]["dtc_trust_scores"] = get_trust_scores(mlcontext.iter_train_X[i].values, mlcontext.iter_train_y[i], mlcontext.iter_test_X[i].values, eval_predict.reshape(-1, 1))

In [ ]:
len(mlcontext.iter_objs[i]["dtc_trust_scores"])

In [ ]:
mlcontext.iter_train_y[i]["Risk Level"].unique()

In [ ]:
mlcontext.iter_train_y[i].shape